In [1]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise import Reader, Dataset
from surprise import KNNBaseline
from surprise.model_selection import GridSearchCV

from surprise import SVD, BaselineOnly, CoClustering

from tqdm import tqdm
import pickle
import itertools

In [2]:
train_group = pd.read_pickle('train_group_add_cookie.pickle')

In [3]:
user_id=train_group.index.get_level_values(0)
vacancy_id_=train_group.index.get_level_values(1)
data = pd.DataFrame(list(zip( user_id, vacancy_id_, train_group['event_type'].to_list())), columns = ['user_id', 'vacancy_id_', 'rating'])

data

,user_id,vacancy_id_,rating
0,000089d26cdd49d68839c68bc10f2cf2,177380,1
1,000089d26cdd49d68839c68bc10f2cf2,181353,1
2,000089d26cdd49d68839c68bc10f2cf2,195890,1
3,000089d26cdd49d68839c68bc10f2cf2,205562,1
4,000089d26cdd49d68839c68bc10f2cf2,211064,1
...,...,...,...
4662685,ffffdb17f45b4032b386d691d52e6c00,107125,10
4662686,ffffdb17f45b4032b386d691d52e6c00,107893,10
4662687,ffffdb17f45b4032b386d691d52e6c00,150843,10
4662688,ffffdb17f45b4032b386d691d52e6c00,160164,10


In [26]:
min_len=[0,5,10,15,20, 30, 40, 50, 100]
max_len=[5000, 200, 150, 100, 50, 40, 30, 20, 10, 5]

df=data.groupby('user_id')['vacancy_id_']. nunique ()

reader = Reader(rating_scale=(0, 10))


bsl_options = {'method': 'als', 
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

all_data_ = Dataset.load_from_df(data[['user_id','vacancy_id_', 'rating']], reader)
all_trainset, all_testset = train_test_split(all_data_, test_size=0.5)

list_mae=[]
list_rmse=[]

list_i_j=[]

all_list_mae=[]
all_list_rmse=[]

for i,j in tqdm(itertools.product(min_len, max_len)):
    data_gd=data.loc[data['user_id'].isin(list(df[(df>=i) & (df<=j)].index))]
    data_ = Dataset.load_from_df(data_gd[['user_id','vacancy_id_', 'rating']], reader)
    
    if len(data_gd)>0:
        trainset, testset = train_test_split(data_, test_size=0.25)
        algo = BaselineOnly(bsl_options=bsl_options)
        predictions = algo.fit(trainset).test(testset)
        list_mae.append(accuracy.mae(predictions))
        list_rmse.append(accuracy.rmse(predictions))
        list_i_j.append((i,j))
    
        predictions = algo.test(all_testset)
        all_list_mae.append(accuracy.mae(predictions))
        all_list_rmse.append(accuracy.rmse(predictions))
    print(i,j)

0it [00:00, ?it/s]

Estimating biases using als...
MAE:  1.9671
RMSE: 2.9026
MAE:  1.8755


1it [01:44, 104.86s/it]

RMSE: 2.7677
0 5000
Estimating biases using als...
MAE:  1.9701
RMSE: 2.9090
MAE:  1.8951


2it [03:33, 107.09s/it]

...

RMSE: 3.2734
100 150
Estimating biases using als...
MAE:  2.1949
RMSE: 3.0489
MAE:  2.7601


86it [1:00:12, 11.02s/it]

RMSE: 3.3237
100 100
100 50
100 40
100 30


90it [1:00:13, 40.15s/it]

100 20
100 10
100 5


In [27]:
accurace_df = pd.DataFrame(list(zip( list_i_j, list_rmse, list_mae, all_list_rmse, all_list_mae)), 
                           columns = ['limitation', 'rmse', 'mae', 'all_rmse', 'all_mae'])

accurace_df 

,limitation,rmse,mae,all_rmse,all_mae
0,"(0, 5000)",2.902578,1.967067,2.767700,1.875531
1,"(0, 200)",2.908983,1.970100,2.801407,1.895131
2,"(0, 150)",2.909407,1.968684,2.810822,1.900435
3,"(0, 100)",2.902261,1.962380,2.831374,1.914382
4,"(0, 50)",2.897146,1.949218,2.899247,1.954566
...,...,...,...,...,...
62,"(50, 50)",2.945060,2.074809,3.281419,2.498582
63,"(100, 5000)",2.959994,2.044912,3.280537,2.692802
64,"(100, 200)",3.007853,2.086045,3.268255,2.546311
65,"(100, 150)",3.015862,2.084946,3.273359,2.529709


In [33]:
accurace_df[accurace_df.rmse<2.9]

,limitation,rmse,mae,all_rmse,all_mae
4,"(0, 50)",2.897146,1.949218,2.899247,1.954566
5,"(0, 40)",2.892206,1.939030,2.932273,1.972208
6,"(0, 30)",2.878798,1.918360,2.981251,1.994931
7,"(0, 20)",2.863191,1.880283,3.056369,2.022330
8,"(0, 10)",2.787610,1.748618,3.188536,2.032015
9,"(0, 5)",2.508766,1.409393,3.306899,1.944424
14,"(5, 50)",2.894907,1.948205,2.898930,1.954256
15,"(5, 40)",2.896067,1.942296,2.932092,1.971873
16,"(5, 30)",2.889746,1.922775,2.981141,1.992252
17,"(5, 20)",2.862019,1.880186,3.056056,2.022203


In [30]:
accurace_df[accurace_df.all_rmse<2.9]

,limitation,rmse,mae,all_rmse,all_mae
0,"(0, 5000)",2.902578,1.967067,2.767700,1.875531
1,"(0, 200)",2.908983,1.970100,2.801407,1.895131
2,"(0, 150)",2.909407,1.968684,2.810822,1.900435
3,"(0, 100)",2.902261,1.962380,2.831374,1.914382
4,"(0, 50)",2.897146,1.949218,2.899247,1.954566
10,"(5, 5000)",2.904775,1.969407,2.768101,1.875460
11,"(5, 200)",2.906327,1.968918,2.801060,1.895370
12,"(5, 150)",2.906871,1.966779,2.810757,1.900647
13,"(5, 100)",2.903472,1.962919,2.831484,1.913973
14,"(5, 50)",2.894907,1.948205,2.898930,1.954256


лучшее значение :

In [32]:
accurace_df.loc[10, ]

limitation    (5, 5000)
rmse           2.904775
mae            1.969407
all_rmse       2.768101
all_mae         1.87546
Name: 10, dtype: object